# Train a scikit-learn SVM on the Iris dataset.

**Notebook Flow** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create and run a `Command` which executes a Python command
- Use a local file as an `input` to the Command

**Motivations** - This notebook explains how to setup and run a Command. The Command is a fundamental construct of Azure Machine Learning. It can be used to run a task on a specified compute (either local or on the cloud). The Command accepts `environment` and `compute` to setup required infrastructure. You can define a `command` to run on this infrastructure with `inputs`.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [2]:
# Enter details of your AML workspace
subscription_id = "f9b97038-ed78-4a26-a1a7-51e81e75d867"
resource_group = "openaml"
workspace = "nlp-workspace"

In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Connect to Compute for training

In [4]:
from azure.ai.ml.entities import AmlCompute

gpu_cluster = ml_client.compute.get("gpu-cluster")
print(
         f" We will use cluster named {gpu_cluster.name} details: {gpu_cluster}"
    )

 We will use cluster named gpu-cluster details: AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'gpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/resourceGroups/openaml/providers/Microsoft.MachineLearningServices/workspaces/nlp-workspace/computes/gpu-cluster', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/eneros3/code/Users/eneros/azureml-poc/multimodel-registry', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f49d6caf640>, 'resource_id': None, 'location': 'eastus2', 'size': 'STANDARD_NC24ADS_A100_V4', 'min_instances': 0, 'max_instances': 1, 'idle_time_before_scale_down': 120.0, 'identity': <azure.ai.ml.entities._credentials.IdentityConfiguration object at 0x7f49d6caee90>, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': Non

# 2. Configure and run the Command
In this section we will configure and run a standalone job using the `command` class. The `command` class can be used to run standalone jobs and can also be used as a function inside pipelines.

## 2.1 Configure the Command
The `command` allows user to configure the following key aspects.
- `code` - This is the path where the code to run the command is located
- `command` - This is the command that needs to be run
- `inputs` - This is the dictionary of inputs using name value pairs to the command. The key is a name for the input within the context of the job and the value is the input value. Inputs can be referenced in the `command` using the `${{inputs.<input_name>}}` expression. To use files or folders as inputs, we can use the `Input` class. The `Input` class supports three parameters:
    - `type` - The type of input. This can be a `uri_file` or `uri_folder`. The default is `uri_folder`.         
    - `path` - The path to the file or folder. These can be local or remote files or folders. For remote files - http/https, wasb are supported. 
        - Azure ML `data`/`dataset` or `datastore` are of type `uri_folder`. To use `data`/`dataset` as input, you can use registered dataset in the workspace using the format '<data_name>:<version>'. For e.g Input(type='uri_folder', path='my_dataset:1')
    - `mode` - 	Mode of how the data should be delivered to the compute target. Allowed values are `ro_mount`, `rw_mount` and `download`. Default is `ro_mount`
- `environment` - This is the environment needed for the command to run. Curated or custom environments from the workspace can be used. Or a custom environment can be created and used as well. Check out the [environment](../../../../assets/environment/environment.ipynb) notebook for more examples.
- `compute` - The compute on which the command will run. In this example we are using a compute called `cpu-cluster` present in the workspace. You can replace it any other compute in the workspace. You can run it on the local machine by using `local` for the compute. This will run the command on the local machine and all the run details and output of the job will be uploaded to the Azure ML workspace.
- `distribution` - Distribution configuration for distributed training scenarios. Azure Machine Learning supports PyTorch, TensorFlow, and MPI-based distributed training. The allowed values are `PyTorch`, `TensorFlow` or `Mpi`.
- `display_name` - The display name of the Job
- `description` - The description of the experiment

In [5]:
# create the command
job = command(
    code="./training",  # local path where the code is stored
    command="python main.py --iris-csv ${{inputs.iris}} --C ${{inputs.C}} --kernel ${{inputs.kernel}} --coef0 ${{inputs.coef0}}",
    inputs={
        "iris": Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",
        ),

        "C": 1.0,
        "kernel": "poly",
        "coef0": 0.1,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="gpu-cluster",
    display_name="sklearn-iris-example",
    experiment_name="iris_model_mlflow_prediction",
    # description
)

## 2.2 Run the Command
Using the `MLClient` created earlier, we will now run this Command as a job in the workspace.

In [6]:
# submit the command
returned_job = ml_client.create_or_update(job)

Uploading training (0.0 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3866/3866 [00:00<00:00, 273503.12it/s]


In [7]:
# monitor the job

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_job.name)

# refresh the latest status of the job after streaming
returned_job = ml_client.jobs.get(name=returned_job.name)

RunId: bold_pen_yw81yxdb3l
Web View: https://ml.azure.com/runs/bold_pen_yw81yxdb3l?wsid=/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/resourcegroups/openaml/workspaces/nlp-workspace

Execution Summary
RunId: bold_pen_yw81yxdb3l
Web View: https://ml.azure.com/runs/bold_pen_yw81yxdb3l?wsid=/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/resourcegroups/openaml/workspaces/nlp-workspace



Bad pipe message: %s [b'(\x04\xeeD\xda\x01\x85\xb9\x07\xc9\xbe \x8f \xd4\xfa?']
Bad pipe message: %s [b'\x03\xe2\xedz\xa7\xc6\x14\xab8\xd5t*\xc1)\xe3\xd9\x12Ya\xab\xcf\xbc\xbe\xc6+\x18\xd6\x1e.\x89\xaf)\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 :%\xe4\xc2\xd2\x94mQ\x11\x82\x9e\xff\xd8\x07\xccu\xa3\xa3\xaf\x08ti0\xa6\xd8\x10\xc14}]']
Bad pipe message: %s [b'L:\x8e\xa6M\xba.\xb0\xee\x96\xa7\xd6\x9a\x16\xafgfG H']
Bad pipe message: %s [b'g\x10\xa7\xdd\xad\x9e\xfc8\xa1\xee`\xfe\x05\xb1\x82&\x06\xe6\x1d\xbe/\x9c\x19\x92\xe6\xf1!\xc3y\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\

# Next Steps
You can see further examples of running a job [here](../../../single-step/)